In [1]:
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_ollama import ChatOllama
from langchain.tools import BaseTool
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_core.callbacks import BaseCallbackHandler
from langchain_tavily import TavilySearch

load_dotenv()

/Users/jeyaprakashrajagopal/vscode_projects/Langchain-Projects/.venv/lib/python3.13/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
/Users/jeyaprakashrajagopal/vscode_projects/Langchain-Projects/.venv/lib/python3.13/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


True

In [2]:
from typing import Any
from uuid import UUID
from langchain_core.outputs import LLMResult

class AgentCallbackHandler(BaseCallbackHandler):
    def on_llm_start(
            self,
            serialized: dict[str, Any],
            prompts: list[str],
            *,
            run_id: UUID,
            parent_run_id: UUID | None = None,
            tags: list[str] | None = None,
            metadata: dict[str, Any] | None = None,
            **kwargs: Any,
        ) -> Any:
            """Run when LLM starts running.

            !!! warning
                This method is called for non-chat models (regular LLMs). If you're
                implementing a handler for a chat model, you should use
                `on_chat_model_start` instead.

            Args:
                serialized: The serialized LLM.
                prompts: The prompts.
                run_id: The run ID. This is the ID of the current run.
                parent_run_id: The parent run ID. This is the ID of the parent run.
                tags: The tags.
                metadata: The metadata.
                **kwargs: Additional keyword arguments.
            """
            print(f"Prompt to llm was: ***\n{prompts[0]}")
            print("******")
    
    def on_llm_end(
        self,
        response: LLMResult,
        *,
        run_id: UUID,
        parent_run_id: UUID | None = None,
        **kwargs: Any,
    ) -> Any:
        """Run when LLM ends running.

        Args:
            response: The response which was generated.
            run_id: The run ID. This is the ID of the current run.
            parent_run_id: The parent run ID. This is the ID of the parent run.
            **kwargs: Additional keyword arguments.
        """
        print(f"***LLM Response***:***\n{response.generations[0][0].text}")
        print("******")

In [3]:
from typing import List

def find_tool_by_name(tools: List[BaseTool], tool_name) -> BaseTool:
    for tool_1 in tools:
        if tool_name == tool_1.name:
            return tool_1
    return ValueError(f"Tool with the name {tool_name}couldn't be found ")

In [ ]:
tools = [TavilySearch()]
llm = ChatOllama(model="gpt-oss:20b", temperature=0, callbacks=[AgentCallbackHandler()])
llm_with_tools = llm.bind_tools(tools=tools)

messages = [
    HumanMessage(content="What is the weather in frankfurt?")
]

while True:
    ai_message = llm_with_tools.invoke(messages)
    messages.append(ai_message)

    tool_calls = getattr(ai_message, "tool_calls", None) or []
    
    if len(tool_calls) > 0:
        for tool_call in tool_calls:
            tool_name = tool_call.get("name")
            tool_args = tool_call.get("args", {})
            tool_call_id = tool_call.get("id")

            tool_to_use = find_tool_by_name(tools, tool_name)
            observation = tool_to_use.invoke(tool_args)
            messages.append(ToolMessage(
                content=str(observation),
                tool_call_id=tool_call_id
            ))
        continue
    
    print(ai_message.content)
    
    break

Prompt to llm was: ***
Human: What is the weather in frankfurt?
******
***LLM Response***:***

******
Prompt to llm was: ***
Human: What is the weather in frankfurt?
AI: [{'name': 'tavily_search', 'args': {'include_images': False, 'query': 'current weather in Frankfurt', 'search_depth': 'basic'}, 'id': '398763c8-a3b3-4e49-b0e4-74f15c4a6b53', 'type': 'tool_call'}]
Tool: {'query': 'current weather in Frankfurt', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Frankfurt', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Frankfurt', 'region': 'Hessen', 'country': 'Germany', 'lat': 50.1167, 'lon': 8.6833, 'tz_id': 'Europe/Berlin', 'localtime_epoch': 1767349716, 'localtime': '2026-01-02 11:28'}, 'current': {'last_updated_epoch': 1767348900, 'last_updated': '2026-01-02 11:15', 'temp_c': 3.3, 'temp_f': 37.9, 'is_day': 1, 'condition': {'text': 'Patchy heavy snow', 'icon': '//cdn.weatherapi.com/weather/64x64/day/335.png', 'code